### Authors:

Marek Szydłowski 136633<br>
Wojciech Tomczak 145467

Link to the source code: https://github.com/Vosloo/evolutionary-computation-labs

In [ ]:
import functools
import pickle
import sys
from itertools import repeat
from multiprocessing import Pool
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Markdown, display

In [ ]:
module_path = Path.cwd().parent
if module_path not in sys.path:
    sys.path.append(str(module_path))

In [ ]:
from src.algorithms.methods import Method
from src.model import Grade
from src.model.enum import Dataset
from src.tsp_problem import TSPProblem
from src.utils import get_edges
from src.visualization import Visualizer

In [ ]:
problem = TSPProblem(no_runs=20)

In [ ]:
def multiprocessing_transform(path: str, dataset: Dataset) -> list[dict[str, dict[str, Grade]]]:
    grades_dump = pickle.load(open(path, "rb"))

    best_run = None
    runs = []
    runtime = 0

    for part_grade in grades_dump:
        grade: Grade = part_grade[dataset][Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM]

        runs.extend(grade.runs)
        runtime += grade._runtime
        if best_run is None or best_run > grade.best_run:
            best_run = grade.best_run

    final_grade = Grade(Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM.value, best_run, runs)
    final_grade.set_runtime(runtime)

    return {dataset: {Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM: final_grade}}

In [ ]:
grades_tspc_lab_8 = multiprocessing_transform("../lab-8/grades_tspc.pkl", Dataset.TSPC.value)
grades_tspd_lab_8 = multiprocessing_transform("../lab-8/grades_tspd.pkl", Dataset.TSPD.value)

In [ ]:
kwargs = {
    "initial_runs": {
        Dataset.TSPC.value: grades_tspc_lab_8[Dataset.TSPC.value][
            Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM
        ].runs,
        Dataset.TSPD.value: grades_tspd_lab_8[Dataset.TSPD.value][
            Method.LOCAL_SEARCH_GREEDY_EDGES_RANDOM
        ].runs,
    }
}

In [ ]:
items = [
    ((Dataset.TSPC.value,), [Method.HYBRID_EVOLUTION_OPERATOR_1], (kwargs)),
    ((Dataset.TSPC.value,), [Method.HYBRID_EVOLUTION_OPERATOR_1_NO_LS], (kwargs)),
    ((Dataset.TSPC.value,), [Method.HYBRID_EVOLUTION_OPERATOR_2], (kwargs)),
    ((Dataset.TSPC.value,), [Method.HYBRID_EVOLUTION_OPERATOR_2_NO_LS], (kwargs)),
    ((Dataset.TSPD.value,), [Method.HYBRID_EVOLUTION_OPERATOR_1], (kwargs)),
    ((Dataset.TSPD.value,), [Method.HYBRID_EVOLUTION_OPERATOR_1_NO_LS], (kwargs)),
    ((Dataset.TSPD.value,), [Method.HYBRID_EVOLUTION_OPERATOR_2], (kwargs)),
    ((Dataset.TSPD.value,), [Method.HYBRID_EVOLUTION_OPERATOR_2_NO_LS], (kwargs)),
]

In [ ]:
with Pool() as pool:
    grades = pool.starmap(problem.run, items)

pickle.dump(grades, open("grades.pkl", "wb"))